Hello. This is my first competition.

Credit to @ulrich07 and @mlconsult - I simply took their notebook and filtered out all but 2021 May and Apr player targets and then medianed for each player.

Here is their noteboook that I forked.

https://www.kaggle.com/ulrich07/baseline-model-player-mean-or-median
https://www.kaggle.com/mlconsult/baseline-average-1-47

In [ ]:
import pandas as pd
import os
import gc
from tqdm import tqdm

import mlb
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

In [ ]:
ROOT_DIR = "../input/mlb-player-digital-engagement-forecasting"

In [ ]:
# Read train.csv
tr = pd.read_csv(f"{ROOT_DIR}/train.csv")
print(tr.shape)
gc.collect()

In [ ]:
tr.head()

## COMPUTING PLAYER MEAN

In [ ]:
# Create list from nextDayPlayerEngagement column
N_DATES = tr.shape[0]
d = []
for idx in tqdm(range(N_DATES)):
    u = eval(tr.iloc[idx, 1])
    d += u
#================

In [ ]:
# Create DataFrame from list
tgt_df = pd.DataFrame(d)
print(tgt_df.shape)

In [ ]:
# Add year column
tgt_df['year'] = pd.DatetimeIndex(tgt_df['engagementMetricsDate']).year
tgt_df['month'] = pd.DatetimeIndex(tgt_df['engagementMetricsDate']).month

In [ ]:
tgt_df

In [ ]:
# Select only 2021
new_df = tgt_df[tgt_df['year'] == 2021]
new_df = new_df[new_df['month'] >= 4]
new_df

In [ ]:
### group by playerId setting median
player_mean = new_df.groupby(["playerId"])[["target1","target2","target3","target4"]].median().reset_index()
gc.collect()
print(player_mean.shape)

In [ ]:
player_mean.head()

In [ ]:
def process_pred(df):
    df["playerId"] = df["date_playerId"].apply(lambda x: int( x.split("_")[1] ) )
    df.drop(["target1","target2","target3","target4"], axis=1, inplace=True)
    df = df.merge(player_mean, on="playerId", how="left")
    df.drop("playerId", axis=1, inplace=True)
    df = df.fillna(0.)
    return df
#===================

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df = process_pred(sample_prediction_df)  # Create prediction
    env.predict(sample_prediction_df)                          # Submit

In [ ]:
sample_prediction_df.head()